# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
import json
from collections import OrderedDict
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNewer.json'])

taggerNewer.json


We set up histograms for each of the channels, and then fill them with tagger times from our data:

In [4]:
taggerChannels = aqdet.Channels['Tagger']     # number of channels in the tagger (368)

histos = [None]*taggerChannels                # set up an array of 368 histograms to fill      

for i in range(taggerChannels):               # for each tagger channel
    histos[i] = Hist(1300,-500,800)           # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)              # calibrate the data
    taggerTimes = aqdet.Arrays['Tagger']['Time']     # get the tagger times
    
    for dat in taggerTimes:                          # for each data point
        for time in dat['Time']:                     # for each tagger time
            for i in range(taggerChannels):          # for each channel
                if (dat['channel']==i):              # if the channel of that data point is the current channel
                    histos[i].Fill(time)             # then put that time in the histogram
                    
    if(aq.eventNo%5000==0):                          # print a processing statement every 5000 events
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,500000)               # run this process over 500000 events

('Events Processed: ', 5000)
('Events Processed: ', 10000)
('Events Processed: ', 15000)
('Events Processed: ', 20000)
('Events Processed: ', 25000)
('Events Processed: ', 30000)
('Events Processed: ', 35000)
('Events Processed: ', 40000)
('Events Processed: ', 45000)
('Events Processed: ', 50000)
('Events Processed: ', 55000)
('Events Processed: ', 60000)
('Events Processed: ', 65000)
('Events Processed: ', 70000)
('Events Processed: ', 75000)
('Events Processed: ', 80000)
('Events Processed: ', 85000)
('Events Processed: ', 90000)
('Events Processed: ', 95000)
('Events Processed: ', 100000)
('Events Processed: ', 105000)
('Events Processed: ', 110000)
('Events Processed: ', 115000)
('Events Processed: ', 120000)
('Events Processed: ', 125000)
('Events Processed: ', 130000)
('Events Processed: ', 135000)
('Events Processed: ', 140000)
('Events Processed: ', 145000)
('Events Processed: ', 150000)
('Events Processed: ', 155000)
('Events Processed: ', 160000)
('Events Processed: ', 16500

Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them:

In [8]:
ROOT.enableJSVis()

tagger_times = ROOT.TFile("tagger_times.root","RECREATE")     # create a root file to put all the histograms into

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")                   # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","background")     # label the parameters

peak = [None]*taggerChannels          # set up an array for the peak locations
sigma = [None]*taggerChannels         # set up an array for the standard deviations of the peaks
bgLevel = [None]*taggerChannels       # set up an array for the background levels

pk = 140     # initialize a peak value to try
sig = 1.1    # initialize a standard deviation value to try
bg = 100     # initialize a background value to try

for i in range(taggerChannels):                                        
    myFit.SetParameter(1,pk)                 
    myFit.SetParameter(2,sig)                
    myFit.SetParameter(3,bg)                
    
    histos[i].SetTitle("Channel "+str(i))      
    histos[i].Draw()                
    
    if(histos[i].GetEntries()>10):                                     
        histos[i].Fit('myFit','SQ','',-300,600)                           
        peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
        sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
        bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
        
        k=1
        while(ROOT.gMinuit.fCstatu!="CONVERGED " or peak[i] > 160 or peak[i] < 120 or sigma[i]<0 or sigma[i]>4):                                     
            myFit.SetParameter(1,140-k)                 
            myFit.SetParameter(2,1)                   
            myFit.SetParameter(3,80-k)
            
            peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
            sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
            bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
            
            k+=1
            print("loop",i,k,peak[i],sigma[i],bgLevel[i])
            
       
        if (sigma[i]<0):
            print("Check channel "+ str(i)+" for sigma")
            print(i,k,pk,sig,bg)
            
   #         
    else:                                            
        print("Histogram "+str(i)+" is empty (has "+str(histos[i].GetEntries())+" entries).")   
     
    histos[i].Write("tagger_times")    

Check channel 8 for sigma
(8, 1, 140, 1.1, 100)
Check channel 10 for sigma
(10, 1, 140, 1.1, 100)
('loop', 11, 2, 139.0, -0.021201922937609717, 53.27355730599798)
('loop', 11, 3, 140.9616036599027, -1.0306721103576022, 53.160180055666274)
Check channel 11 for sigma
(11, 3, 140, 1.1, 100)
('loop', 12, 2, 141.1944316143969, -0.9904138587533782, 52.39169573257253)
Check channel 12 for sigma
(12, 2, 140, 1.1, 100)
Check channel 13 for sigma
(13, 1, 140, 1.1, 100)
('loop', 14, 2, 139.0, 0.017053716180727194, 45.36612452607346)
Check channel 15 for sigma
(15, 1, 140, 1.1, 100)
Check channel 17 for sigma
(17, 1, 140, 1.1, 100)
Check channel 30 for sigma
(30, 1, 140, 1.1, 100)
Check channel 38 for sigma
(38, 1, 140, 1.1, 100)
Check channel 73 for sigma
(73, 1, 140, 1.1, 100)
Check channel 74 for sigma
(74, 1, 140, 1.1, 100)
Check channel 76 for sigma
(76, 1, 140, 1.1, 100)
('loop', 77, 2, 137.32037454826133, 1.051829889193823, 9.702951616189454)
Check channel 78 for sigma
(78, 1, 140, 1.1, 100

('loop', 203, 2, 155.79678266071022, 0.012390472903965326, 2.249300059552261)
('loop', 203, 3, 129.19466869257917, -1.0312283714021337, 2.235646192157536)
Check channel 203 for sigma
(203, 3, 140, 1.1, 100)
('loop', 204, 2, 137.88444524978647, 0.039347752777776135, 2.1931371207431707)
('loop', 204, 3, 138.45854319412493, 0.17189363205216704, 2.191268257268705)
('loop', 204, 4, 138.41078217504378, -0.47234067614710884, 2.191260335320661)
Check channel 204 for sigma
(204, 4, 140, 1.1, 100)
('loop', 206, 2, 159.8662362870244, -48.07159986639466, 2.2188550019305353)
('loop', 206, 3, 367.1524164491137, 20289.470627910185, -766.4223517563332)
('loop', 206, 4, 136.39077733268587, 0.9627493016313053, 2.190714791261126)
Check channel 207 for sigma
(207, 1, 140, 1.1, 100)
Check channel 208 for sigma
(208, 1, 140, 1.1, 100)
('loop', 209, 2, 141.948841814385, -1.3585632894150486, 2.2010298123974796)
Check channel 209 for sigma
(209, 2, 140, 1.1, 100)
('loop', 210, 2, 137.79994029737844, 0.08469768

('loop', 262, 2, 120.7704199798173, 2.186221235949961, 1.8116848104193657)
Check channel 263 for sigma
(263, 1, 140, 1.1, 100)
('loop', 264, 2, 137.71091783204022, 0.1662327910011509, 1.7761141658291884)
('loop', 264, 3, 137.79121867630747, 0.15354250587906773, 1.7761129716671398)
('loop', 264, 4, 138.90353710481645, -4.386756940174213, 1.789301339631376)
('loop', 264, 5, 120.00009515017614, 0.30044317028581413, 1.7738495991038206)
('loop', 264, 6, 135.0, -0.000802426481434193, 1.774359996722039)
('loop', 264, 7, 131.49999782144576, -0.18373565835717712, 1.7778720054357173)
Check channel 264 for sigma
(264, 7, 140, 1.1, 100)
Check channel 265 for sigma
(265, 1, 140, 1.1, 100)
('loop', 266, 2, 119.2992437343105, 9751.166956746649, 164.98248791847348)
('loop', 266, 3, 133.67024937793602, -1.465418618441139, 1.763908158514743)
Check channel 266 for sigma
(266, 3, 140, 1.1, 100)
('loop', 267, 2, 144.14515627913534, 0.2558593244522406, 1.7556631370264435)
('loop', 267, 3, 138.7790853634617,

('loop', 315, 3, 4047721977.989679, 1.0, 1.5118393159586718)
('loop', 315, 4, 137.15204188376842, -0.2738255804277606, 1.5102683984678922)
('loop', 315, 5, 148.13647851801014, -0.20363075061583513, 1.5113183992474093)
('loop', 315, 6, 124.96224401239675, -0.8071584114943905, 1.5048587865582608)
Check channel 315 for sigma
(315, 6, 140, 1.1, 100)
('loop', 316, 2, 161.62384690755422, 0.17011043457352257, 1.5398414636345898)
('loop', 316, 3, 136.94754767346305, 0.00015703869650073206, 1.5409184655956008)
('loop', 316, 4, 137.13732072237767, 0.0972606108129441, 1.5404135139623172)
('loop', 316, 5, 135.06139377761386, 0.07026604363282273, 1.5398426314639948)
('loop', 316, 6, 135.0631974160217, 0.07149558693261433, 1.539842705656643)
('loop', 316, 7, 135.14768168900102, 0.059439083926145275, 1.5398424060906153)
('loop', 316, 8, 131.69137018961638, 0.1340642031699549, 1.540407757638338)
('loop', 316, 9, 131.6561911676521, 0.14433372914806658, 1.540407198327335)
('loop', 316, 10, 134.959671593

Check channel 352 for sigma
(352, 3, 140, 1.1, 100)
Check channel 353 for sigma
(353, 1, 140, 1.1, 100)
('loop', 354, 2, 144.6376105635698, -1.3178220504108162, 1.782885364080363)
Check channel 354 for sigma
(354, 2, 140, 1.1, 100)
('loop', 355, 2, 137.9999987904087, 0.3007820786215722, 1.6514865416081908)
('loop', 355, 3, 137.9999922496379, 0.34608915857004036, 1.6514857845842301)
('loop', 355, 4, 138.0000002952469, 0.35045815874167324, 1.6514857752733791)
Check channel 356 for sigma
(356, 1, 140, 1.1, 100)
('loop', 357, 2, 140.09016421065633, 0.0807030750752624, 1.631095244244444)
('loop', 357, 3, 136.82959917653216, 0.09208493790093929, 1.6283485522425714)
('loop', 357, 4, 136.78781126217223, 0.20307482444396346, 1.628350258517825)
('loop', 357, 5, 136.91433120383687, -0.10556265933020126, 1.628348297243562)
('loop', 357, 6, 133.49996907302432, -0.1731385039827567, 1.6288473492890054)
Check channel 357 for sigma
(357, 6, 140, 1.1, 100)
('loop', 358, 2, 144.3976196507115, 0.016535473

Finding how much each peak is off from zero, and setting it back to zero:

In [6]:
bins = 1300
ROOT.enableJSVis()
tagger_calib = ROOT.TFile("tagger_calib.root","RECREATE")

histos_calib = [None]*taggerChannels
for i in range(taggerChannels):
    histos_calib[i] = Hist(bins,-500,800)

for i in range(taggerChannels):
    print(i, peak[i])
    if (peak[i]!=None):
        histos_calib[i] = Hist(bins,int(-500-peak[i]),int(800-peak[i]))
        for j in range(bins):
            histos_calib[i].SetBinContent(j,histos[i].GetBinContent(j))
     
    histos_calib[i].SetTitle("Channel "+str(i))      
    histos_calib[i].Draw() 
    
    histos_calib[i].Write("tagger_calib")  

(0, 141.4042947270836)
(1, 142.28949720528084)
(2, 141.94182736480454)
(3, 141.3305394562416)
(4, 143.88012519836644)
(5, 142.23938725218036)
(6, 142.38047226689775)
(7, 141.87718539307477)
(8, 141.4617262172606)
(9, 142.55642898602932)
(10, 141.54571877439778)
(11, 141.00716798257548)
(12, 141.15864868655183)
(13, 141.52354248405885)
(14, 142.15461158336666)
(15, 142.21064023193884)
(16, 143.7957869262562)
(17, 144.400522965579)
(18, 143.29396557574674)
(19, 143.1916967315927)
(20, 143.15872256395076)
(21, 143.59149619792203)
(22, 143.910891686622)
(23, 143.88712718343433)
(24, 140.8918319382066)
(25, 134.46077710350292)
(26, 140.44715947976331)
(27, 140.38135524366072)
(28, 140.95142945661215)
(29, 141.08628247401242)
(30, 141.7166733939442)
(31, 142.8923939065237)
(32, 140.2949200298721)
(33, 141.194192692386)
(34, 139.7783838412325)
(35, 139.8789342011421)
(36, 139.97978754273674)
(37, 140.4580655040984)
(38, 143.87821333210061)
(39, 140.60819683363047)
(40, 141.43044373584914)
(41

(358, 145.17858719244023)
(359, 145.64072668023275)
(360, 142.23190934109644)
(361, 143.61377268683137)
(362, 143.38021878753727)
(363, 143.03267335511663)
(364, 142.73433358522513)
(365, 143.85387624109123)
(366, 143.227516458119)
(367, 143.47019667920063)


Next, we must import the json file, change the channel offset values, and then save the array again as a new json file. Note that the empty histograms write a value of 'None' to the json file for their offsets.

In [14]:
calibData = json.load(open('taggerNewer.json'), object_pairs_hook=OrderedDict)     # load the tagger calibration data and keep its original ordering

i = 0                                   # starting at channel 0
for chan in calibData['channels']:      # for each of the tagger channels
    chan['offset'][0] = peak[i]         # change the offset value to the one found in our earlier fit for that channel
    i+=1                                # move on to the next channel

json.dump(calibData, open('taggerWithOffsets.json', 'w'), indent = 2)              # write the changed data to a new json file